### Instructions: Follow the instructions provided in each step, or in the output from a cell 
Step 1
* Enter the name of the pivot table in the next cell, then press the 'run' button above.
* If you see the text 'Everything looks good!' beneath the cell after you press the run button, scan the column names to verify that they are correct.  If they are, skip the following cells and proceed to Step 2.
* Otherwise, follow the instructions in the output.

In [2]:
pivot_table = '20411_Felber1-2_MHC-I_Haplotypes_23Mar18.xlsx' # enter name here

In [9]:
import pandas as pd
from pandas import DataFrame
import re

def parseExcelWithPandas(fName):
    eSheetData = ''
    dfs = {sheet_name: testFile.parse(sheet_name)
      for sheet_name in testFile.sheet_names}
    if not fName:
        sheetCount = -1
        for s in dfs:
            sheetCount += 1
            m = re.search('pivot', s)
            m2 = re.search('MiSeq', s)
            if m:
                # JRC Comment:
                # using pd.read_excel(pivot_table, sheet_name=excelSheetName)
                # is vulnerable to a bug whereby pandas will intermittently ignore parameters and grab the first sheet only, 
                # UNLESS the 0-indexed integer for the sheet is specified with the function x = pd.ExcelFile() followed by x.parse(int)
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            elif m2:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    else:
        for s in dfs:
            m = re.search(fName, s) # case insensitive?
            if m:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    return eSheetData
testFile = pd.ExcelFile(pivot_table)
readyToProceed = False
fName_none = ''
dataAsPandas = ''
excelSheetName = parseExcelWithPandas(fName_none)
processedPivotTable = False
if pivot_table:
    if excelSheetName.empty:
        print('The Excel sheet was found, but there was an error reading the Excel file.')
        print('Please do one of the following:\n\nProceed to the next cell and attempt to enter the sheet_name value,\n\nor\n\nExport the file as a csv, start from the beginning of the python notebook,\nchange the file name, and rerun all cells.\nBe sure to specify the file type as csv in the cell below when you run it.')
    else:
        print('Everything looks good!')
        print('Here is a preview of the data: \n\n##StartPreview:\n')
        pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
        for h in pdHeadersRowsCol1:
            m = re.search('Animal ID', h)
            if m:
                processedPivotTable = True
            print(h + '\n')
        print('##EndPreview\n\nData Type is ')
        if processedPivotTable:
            print('Processed Pivot Table')
        else:
            print('raw pivot table')
        readyToProceed = True
else:
    print('No pivot table was found. Please do the following: \n1) check the filename and rerun the previous cell\n2) if you have already rerun the previous cell and are seeing this message again, \nproceed to the next cell and enter information for at least one of the following: ')
    print('\tfile_type: enter \'csv\' or \'excel\', depending on the file.')
    print('\tsheet_name: enter the sheet name for the excel spreadsheet or csv file')
    print('Then run the next two cells.')

Everything looks good!
Here is a preview of the data: 

##StartPreview:

Animal ID

# Reads Evaluated

# Reads Identified

% Unknown

Mamu-A Haplotype 1

Mamu-A Haplotype 2

Mamu-B Haplotype 1

Mamu-B Haplotype 2

Comments

##EndPreview

Data Type is 
raw pivot table


In [101]:
import time
import math
# print(excelSheetName.columns.values)
def findColumnIdxStartStop(pdDF):
    xCt = -1
    xStart = -1
    xStop = -1
    foundInitialMatch = False
    for x in excelSheetName.columns.values:
        xCt += 1
        if xCt == 0:
            continue
        else:
            if xCt < 10 and foundInitialMatch == False: # column idx will not be greater than 9
                m = re.search('named', str(x))
                if m:
                    continue
                else:
                    foundInitialMatch = True
                    xStart = xCt
            else: 
                m = re.search('named', str(x))
                if m:
                    xStop = xCt
                    break
    return (xStart, xStop)
def parsePandasDfRows(col1ListFromPdDf):
    headers = True
    mamuA_indices = []
    mamuB_indices = []
    skipIndices = []
    genotypeList = []
    for idx,val in enumerate(col1ListFromPdDf):
        if headers:
            m = re.search('Comment', str(val))
            mA = re.search('Mamu-A', str(val))
            mB = re.search('Mamu-B', str(val))
            if m:
                headers = False
                skipIndices.append(idx)
                continue
            elif mA:
                mamuA_indices.append(idx)
            elif mB:
                mamuB_indices.append(idx)
            else:
                skipIndices.append(idx)
                continue
        else:
            m = re.search('Alleles', str(val))
            if m:
                skipIndices.append(idx)
            else:
                genotypeList.append(val)
                continue
    return (skipIndices, mamuA_indices, mamuB_indices, genotypeList)
def dataToPandasOneHot(d, pdDf, idxList):
    if pdDf is None:
        pdDf = pd.DataFrame.from_dict(d, orient='index').transpose()
        pdDf.index = idxList
        pdDf.index.name = 'genotype'
    else:
        alleleDFnew = pd.DataFrame.from_dict(d, orient='index').transpose()
        alleleDFnew.index = idxList
        alleleDFnew.index.name = 'genotype'
        alleleDFres = pd.concat([pdDf, alleleDFnew], axis=1, join_axes=[pdDf.index])
        pdDf = alleleDFres
    return pdDf
v = findColumnIdxStartStop(excelSheetName)
if v[0] != -1:
    print('Start at column ' + str(v[0]))
    print('Stop at column ' + str(v[1]))
else:
    print('Error!  check dataframe!')
col1List = list(excelSheetName['Sample Sheet #'])
skipRows, mamuA_rows, mamuB_rows, genotypeList = parsePandasDfRows(col1List)
skipMamuRows = mamuA_rows + mamuB_rows + skipRows
skipMamuRows.sort()
# for idx,row in excelSheetName.iterrows():
rStart = v[0]
rStop = v[1] + 1 # check this.
mamuA_alleleList = []
mamuB_alleleList = []
'''
        alleleDF = pd.DataFrame.from_dict(pdDict, orient='index').transpose()
        alleleDF.index=allAlleleList
        alleleDF.index.name = 'allele'
'''
alleleDF_MamuA = None
alleleDF_MamuB = None
for x in range(rStart, rStop):
    mamu_genotypes_oneHot = []
    dfValue = excelSheetName.iloc[:,x] # verify syntax for rows and columns, and index not lookup
    dfValue_MamuA_1 = dfValue.iloc[int(mamuA_rows[0])]
    dfValue_MamuA_2 = dfValue.iloc[int(mamuA_rows[1])]
    dfValue_MamuB_1 = dfValue.iloc[int(mamuB_rows[0])]
    dfValue_MamuB_2 = dfValue.iloc[int(mamuB_rows[1])]
    pdDict_MamuA = dict()
    pdDict_MamuB = dict()
    for idx, row in dfValue.iteritems():
        if idx not in skipMamuRows:
            try:
                if math.isnan(row):
                    mamu_genotypes_oneHot.append(0)
                    continue
                else:
                    mamu_genotypes_oneHot.append(1)
                    # this won't work and isn't necessary anyway: need to have 2 linked lists: [alleles] and [0,1,...,1]
                    # aName = col1List[int(idx)]
                    # mamuA_genotypes.append(str(aName))
                    # mamuB_genotypes.append(str(aName))
            except TypeError:
                # WTF, values are entered as '[1]'
                # figure out later if these should be counted as True or False
                # assume True for the moment
                #
                # see above
                # aName = col1List[int(idx)]
                # mamuA_genotypes.append(str(aName))
                # mamuB_genotypes.append(str(aName))
                mamu_genotypes_oneHot.append(1)
            # if math.isnan(row):
            #     continue
            # else:
            #    aName = col1List[int(idx)]
            #    mamuA_genotypes.append(str(aName))
            #    mamuB_genotypes.append(str(aName))
    pdDictKey_MamuA = str(dfValue_MamuA_1) + '-' + str(dfValue_MamuA_2)
    pdDictKey_MamuB = str(dfValue_MamuB_1) + '-' + str(dfValue_MamuB_2)
    pdDict_MamuA[pdDictKey_MamuA] = mamu_genotypes_oneHot
    pdDict_MamuB[pdDictKey_MamuB] = mamu_genotypes_oneHot
    alleleDF_MamuA = dataToPandasOneHot(pdDict_MamuA, alleleDF_MamuA, genotypeList)
    # print(alleleDF_MamuA)
print(alleleDF_MamuA)
    # mamuA_alleleList.append(([str(dfValue_MamuA_1), str(dfValue_MamuA_2)], mamuA_genotypes))
    # mamuB_alleleList.append(([str(dfValue_MamuB_1), str(dfValue_MamuB_2)], mamuB_genotypes))
# for aX in mamuA_alleleList:
    # print('Alleles are ' + str(aX[0][0]) + ',' + str(aX[0][1]))
    # o = ','.join(aX[1])
    # print(o)

        
    
'''
for idx,val in enumerate(col1List):
    if idx in skipRows:
        print('Skipping this: ' + str(val))
    elif idx in mamuA_rows:
        print('Mamu-A value ' + str(val))
    elif idx in mamuB_rows:
        print('Mamu-B value ' + str(val))
    else:
        continue
'''

Start at column 4
Stop at column 78
                                             A008-A025  A008-A025  A016-A023  \
genotype                                                                       
01_Mamu_A1_001g2                                     0          0          0   
01_Mamu_A1_002_01                                    0          0          0   
01_Mamu_A1_002_w_02                                  0          0          0   
01_Mamu_A1_003_02                                    0          0          0   
01_Mamu_A1_003_w_09                                  0          0          0   
01_Mamu_A1_003g1                                     0          0          0   
01_Mamu_A1_004_SP_exp13277_ctg255_47_157bp           0          0          0   
01_Mamu_A1_004g                                      0          0          0   
01_Mamu_A1_006g                                      0          0          0   
01_Mamu_A1_007g1                                     0          0          0   
01_M

"\nfor idx,val in enumerate(col1List):\n    if idx in skipRows:\n        print('Skipping this: ' + str(val))\n    elif idx in mamuA_rows:\n        print('Mamu-A value ' + str(val))\n    elif idx in mamuB_rows:\n        print('Mamu-B value ' + str(val))\n    else:\n        continue\n"

In [74]:
headers = True
mamuA_indices = []
mamuB_indices = []
skipIndices = []
for idx,val in enumerate(col1List):
    if headers:
        m = re.search('Comment', str(val))
        mA = re.search('Mamu-A', str(val))
        mB = re.search('Mamu-B', str(val))
        if m:
            headers = False
            continue
        elif mA:
            mamuA_indices.append(idx)
        elif mB:
            mamuB_indices.append(idx)
        else:
            skipIndices.append(idx)
            continue
    else:
        m = re.search('Alleles', str(val))
        if m:
            skipIndices.append(idx)
        else:
            continue
for idx,val in enumerate(col1List):
    if idx in skipIndices:
        print('Skipping this: ' + str(val))
    elif idx in mamuA_indices:
        print('Mamu-A value ' + str(val))
    elif idx in mamuB_indices:
        print('Mamu-B value ' + str(val))
    else:
        continue



Skipping this: GS ID
Skipping this: Animal ID
Skipping this: # Reads Evaluated
Skipping this: # Reads Identified
Skipping this: % Unknown
Mamu-A value Mamu-A Haplotype 1
Mamu-A value Mamu-A Haplotype 2
Mamu-B value Mamu-B Haplotype 1
Mamu-B value Mamu-B Haplotype 2
Skipping this: Mamu-A1 Alleles
Skipping this: Mamu-A Minor Alleles
Skipping this: Mamu-B Major Alleles
Skipping this: Mamu-B Minor Alleles
Skipping this: Mamu-I Alleles
Skipping this: Psuedogenes and Nonclassical Alleles
Skipping this: Mamu-E Alleles
Skipping this: Mamu-F Alleles
Skipping this: Mamu-AG Alleles
Skipping this: Mamu-G Alleles
Skipping this: Mamu-70 Alleles


In [10]:
file_type = '' # must be left blank, 'csv' for a csv file, or 'excel' for an excel file type
sheet_name = '' # must have the excel sheet name with the pivot table, or be blank

In [12]:
# readyToProceed = False
skipBool = False
if readyToProceed:
    print('It looks like you are running this cell after the data has already been set up,\nso this cell will be skipped.\nIf you are certain that you want to run this cell, uncomment the first line.')
else:
    if sheet_name:
        excelSheetName = parseExcelWithPandas(sheet_name)
        if excelSheetName.empty:
            skipBool = True
            print('Unable to find Excel sheet named ' + str(sheet_name))
        else:
            print('Everything looks good!')
            print('Here is a preview of the data: \n')
            pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
            for h in pdHeadersRowsCol1:
                print(h + '\n')
            readyToProceed = True
    elif file_type:
        if file_type == 'csv':
            csvFileConv = pivot_table.split('.')
            csvFileConvString = '.'.join(csvFileConv[:-1])
            csvFileConvName = csvFileConvString + '.csv'
            try:
                dataAsPandas = pd.read_csv(csvFileConvName)
            except FileNotFoundError:
                print('csv file ' + csvFileConvName + ' not found.\nPlease verify it is named correctly and that it exists.')
                skipBool = True
            except UnicodeDecodeError:
                print('please make sure that the file ' + pivot_table + '\nhas been converted correctly to csv format. ')
                skipBool = True
                dfs = pd.read_excel(file_name, sheet_name=None)
        elif file_type == 'excel':
            excelSheetName = parseExcelWithPandas(sheet_name)
            if excelSheetName.empty:
                skipBool = True
                print('Unable to parse Excel sheet')
            else:
                print('Everything looks good!')
                print('Here is a preview of the data: \n')
                pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
                for h in pdHeadersRowsCol1:
                    print(h + '\n')
                readyToProceed = True
        else:
            print('File Type not recognized.')
            skipBool = True
    else:
        print('It looks like you have not entered any additional data.\nThe workflow cannot proceed unless you specify the file type or if you specify the sheet name, or both.')
if skipBool:
    print('Please re-enter data in the cell above, then rerun that cell and this one.')

It looks like you are running this cell after the data has already been set up,
so this cell will be skipped.
If you are certain that you want to run this cell, uncomment the first line.


In [ ]:
    if alleleDF_MamuA is None:
        alleleDF_MamuA = pd.DataFrame.from_dict(pdDict_MamuA, orient='index').transpose()
        alleleDF_MamuA.index = genotypeList
        alleleDF_MamuA.index.name = 'genotype'
    else:
        alleleDFnew = pd.DataFrame.from_dict(pdDict, orient='index').transpose()
        alleleDFnew.index = genotypeList
        alleleDFnew.index.name = 'genotype'
        alleleDFres = pd.concat([alleleDF_MamuA, alleleDFnew], axis=1, join_axes=[alleleDF_MamuA.index])
        alleleDF = alleleDFres
    if alleleDF_MamuB is None:
        print()
    else:
        print()

Step 2
* If you did not encounter any errors previously, or the output did not direct you to stop, then proceed with the analysis.
* You can either click this cell, select the Cell menu above, and then select 'Run All below', or click the cells after this one and individually click 'Run' for each one
* The analysis within this python notebook will do the following:
  * parse out genotype ID's for the MHC-A, and MHC-B data, 
  * use a pre-trained model for a Machine Learning Classifier to predict the Haplotype for each sample for MHC-A and MHC-B
  * add the predicted values back into the pivot table (or create a new pivot table)
  * output the resulting pivot table
* Note that for 1.x builds, only MHC-A and MHC-B haplotypes will be predicted.  All other haplotypes will be passed to the researcher.  A researcher should also verify the output pivot table from the Classifier.